## SONAR - Binary Classification Project for Navy 

In [1]:
import tensorflow
print(tensorflow.__version__)

2.0.0


In [2]:
# importing classes and functions
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
# fixed random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:
# loading dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# splitting into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]


In [5]:
# converting string values to integer using labelencoder
le = LabelEncoder()
le.fit(Y)
encoded_Y=le.transform(Y)
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
from keras import models
from keras import layers

def create_baseline():
    
    # creating model
    model = models.Sequential()
    model.add(layers.Dense(60, activation='relu', input_shape=(60,)))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compiling model
    model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])

    return model

model = create_baseline()


In [7]:
model.fit(X,
          encoded_Y,
          epochs=20,
          batch_size=5
          )

Epoch 1/20
208/208 [==============================] - 2s 10ms/step - loss: 0.6801 - accuracy: 0.5625
Epoch 2/20
208/208 [==============================] - 0s 341us/step - loss: 0.6205 - accuracy: 0.6923
Epoch 3/20
208/208 [==============================] - 0s 360us/step - loss: 0.5923 - accuracy: 0.7356
Epoch 4/20
208/208 [==============================] - 0s 346us/step - loss: 0.5602 - accuracy: 0.7885
Epoch 5/20
208/208 [==============================] - 0s 365us/step - loss: 0.5452 - accuracy: 0.7212
Epoch 6/20
208/208 [==============================] - 0s 356us/step - loss: 0.5232 - accuracy: 0.7644
Epoch 7/20
208/208 [==============================] - 0s 356us/step - loss: 0.5045 - accuracy: 0.7837
Epoch 8/20
208/208 [==============================] - 0s 365us/step - loss: 0.4916 - accuracy: 0.7740
Epoch 9/20
208/208 [==============================] - 0s 351us/step - loss: 0.4775 - accuracy: 0.7933
Epoch 10/20
208/208 [==============================] - 0s 360us/step - loss: 0.4602

In [8]:
# evaluate model with standardized dataset

estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 82.21% (5.73%)


In [9]:
# evaluate baseline model with standardized dataset

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 84.59% (6.01%)


In [10]:
# smaller model
def create_smaller():
    
    # create model
    model = models.Sequential()
    model.add(layers.Dense(30, activation='relu', input_shape=(60,)))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    return model


In [11]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 83.59% (5.54%)


In [12]:
# larger model
def create_larger():

    # create model
    model = models.Sequential()
    model.add(layers.Dense(60, activation='relu', input_shape=(60,)))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    return model

In [13]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 83.16% (4.42%)


In [16]:
x_val = X[:60]
partial_x_train = X[60:]

y_val = encoded_Y[:60]
partial_y_train = encoded_Y[60:]

In [17]:

# creating model
model = models.Sequential()
model.add(layers.Dense(60, activation='relu', input_shape=(60,)))
model.add(layers.Dense(60, activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compiling model
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

model.fit(partial_x_train,
        partial_y_train,
        epochs=30,
        batch_size=5,
        validation_data=(x_val, y_val))

Train on 148 samples, validate on 60 samples
Epoch 1/30
148/148 [==============================] - 0s 2ms/step - loss: 0.5850 - accuracy: 0.7500 - val_loss: 1.4003 - val_accuracy: 0.0000e+00
Epoch 2/30
148/148 [==============================] - 0s 412us/step - loss: 0.5341 - accuracy: 0.7500 - val_loss: 1.2754 - val_accuracy: 0.0000e+00
Epoch 3/30
148/148 [==============================] - 0s 412us/step - loss: 0.5096 - accuracy: 0.7500 - val_loss: 1.3694 - val_accuracy: 0.0000e+00
Epoch 4/30
148/148 [==============================] - 0s 547us/step - loss: 0.4693 - accuracy: 0.7500 - val_loss: 1.1894 - val_accuracy: 0.0000e+00
Epoch 5/30
148/148 [==============================] - 0s 533us/step - loss: 0.4038 - accuracy: 0.7500 - val_loss: 1.3187 - val_accuracy: 0.2667
Epoch 6/30
148/148 [==============================] - 0s 506us/step - loss: 0.3193 - accuracy: 0.8514 - val_loss: 1.3055 - val_accuracy: 0.3500
Epoch 7/30
148/148 [==============================] - 0s 540us/step - loss: 0

In [25]:
# using the functional API
import keras
from tensorflow.keras import models
from tensorflow.keras import layers

inputs = keras.Input(shape=(60,))
X = layers.Dense(60, activation='relu')(X)
X = layers.Dense(30, activation='relu')(X)
outputs = layers.Dense(1, activation='sigmoid')(X)

model = keras.model(inputs, outputs)

model.fit(X, encoded_Y, epochs = 10, batch_size = 32)

AttributeError: module 'keras' has no attribute 'model'